In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
# 1. 토크나이저와  모델 로드

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B")

base = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-1.7B",
                                            torch_dtype="auto",
                                            device_map="auto")


# 3. 시스템 프롬프트 
system_prompt = """당신은 뉴스 텍스트를 분석하여 카테고리를 분류하고 주요 핵심 이벤트들을 추출하는 전문 분석 시스템입니다.
주어진 텍스트를 분석하여 반드시 파이썬의 dictionary 형식으로 결과를 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지시사항을 그대로 적지 마시고, 해당 지시사항에 따라 적절한 value를 채워넣으십시오.

분석 결과는 다음 형식으로 작성하십시오:

답변:
{"category": "텍스트의 카테고리를 ['부동산', '산업', '오피니언', '증권'] 중 하나로 분류하여 작성하십시오",
"event_count": "텍스트 내에서 발견된 핵심 이벤트의 개수를 정수로 작성하십시오(증감, 변화, 변동이 나오는 구절을 중점적으로 보세요.)",
"events": ["텍스트에서 추출한 각 핵심 이벤트를 요약하는 문장들을 파이썬 문자열 리스트 형태로 작성하십시오. 원문의 핵심 내용을 그대로 유지하되 한 문장으로 간결하게 작성하십시오"]}"""

# 4. 테스트 데이터 3개 
test_texts = [
    # 1. 부동산 카테고리
    """ 최대 수혜지역이라고요? 분위기 좀 달라지긴 했지만, 그게 꼭 6·27 대출 규제 때문만은 아니죠."

지난 19일 서울 도봉구 창동역 인근 한 공인중개사는 최근 시장 분위기를 묻는 질문에 이같이 말했다. 도봉구는 창동 차량기지 복합개발, GTX-C 등 대형 교통 호재에 힘입어 올해 들어 아파트값이 상승세로 전환됐고, 일부 단지에서는 실거래가 회복 조짐도 나타나고 있다. 반면 6·27대책 발표 직후 상승세 강했던 금관구(금천·관악·구로)는 매수·매도인 모두 지켜보기에 들어간 모습이다.

■도봉구 상승세…실거래가 회복 조짐

6·27 대출 규제 시행 3주차에 접어들면서 서울 외곽 지역을 중심으로 시장의 온도차가 뚜렷해지고 있다. 한국부동산원 주간 아파트가격 동향에 따르면 7월 2주(15일 기준) 서울 아파트값은 0.19% 올라 전주(0.29%)보다 오름폭이 둔화됐다. 다만 도봉구(0.05%→0.06%)와 중구(0.16%→0.18%)는 상승폭이 소폭 확대됐다. 도봉구의 한 중개업소 관계자는 "급매가 거의 소진된 상태에서 전고점에 근접한 거래도 하나둘 나타나고 있다"며 "전용 84㎡, 165㎡ 등 다양한 평형에 실수요자와 일부 투자자의 문의가 이어지고 있다"고 전했다. 이어 "6·27 대책 이후 도봉구처럼 상대적으로 대출 여건이 유지되는 중저가 지역에 관심이 모이고 있는 건 사실이지만, 지금 시장을 움직이는 건 결국 개발 기대감에 기반한 유동성 있는 수요자들"이라고 설명했다.

서울부동산정보광장에 따르면 도봉구 아파트 거래량은 5월 155건, 6월 289건으로 늘었지만, 7월 들어서는 20일 기준 51건에 머물고 있다. 규제 여파로 거래량은 다소 숨고르기에 들어간 양상이지만 가격 상승세는 계속될 것으로 전망됐다.

도봉구의 또 다른 중개업소 대표는 "이미 작년에 바닥을 찍었다는 인식이 확산되면서 실수요자 문의가 늘고 있고, 호가도 빠르게 회복되고 있다"며 "GTX-C, 창동 차량기지 개발 등 향후 호재가 여전한 만큼, 지금은 회복의 시작일 뿐이라는 얘기도 나온다"고 했다.

■금관구 숨고르기…거래량 감소 뚜렷

반면 6·27 대책 직후 풍선효과 기대감이 부각됐던 '금관구'는 오름세가 한풀 꺾인 분위기다. 같은 기간 금천구는 0.09%→0.07%, 관악구는 0.19%→0.15%, 구로구는 0.18%→0.12%로 상승폭이 일제히 축소됐다.

금천구 시흥동의 한 중개업소 관계자는 "호가는 잠깐 반짝했지만 지금은 문의가 다시 줄어든 상황"이라며 "갭투자자든 실수요자든 모두 관망 중"이라고 말했다. 관악구의 한 공인중개사도 "전세보증금이 낮은 구축 소형 위주로 간헐적인 갭투자 수요는 있었지만, 지금은 전세금도 안 오르고 매수도 끊긴 상태"라고 분위기를 전했다.

서울부동산정보광장에 따르면 금천구 아파트 거래량은 5월 74건에서 6월 135건으로 증가한 뒤, 7월(20일 기준)에는 25건에 그쳤다. 관악구는 같은 기간 222건→294건→54건, 구로구는 678건→462건→84건으로 줄며 거래 감소세가 뚜렷하다.

개봉동의 중개업소 관계자는 "급매라고는 해도 1000만~2000만원 낮춘 수준인데, 그런 물건도 상반기에 이미 빠졌다"며 "매수자는 관망세, 매도자는 버티기에 들어간 상황"이라고 전했다. 결론적으로 매물 자체가 줄어들면서 거래량도 감소한 위축된 시장 흐름을 보이고 있어, 풍선효과가 나타난다고 보기는 어렵다는 평가다.

부동산업계 한 전문가는 "서울 외곽 지역은 규제 변화에 민감하게 반응하는 편이지만, 신축 비중이나 입지 여건, 실수요자의 선호도에 따라 시장 흐름은 엇갈릴 수밖에 없다"며 "현재는 거래 절벽에 가까운 상태로 '풍선효과' 역시 일부 지역에 국한된 단기 흐름에 그칠 가능성이 높다"고 진단했다.",
""",    
    # 2. 산업 카테고리
    """
    한일시멘트가 자회사 한일현대시멘트 흡수 합병을 통해 경영효율화에 나서면서 시멘트업계에 구조·사업개편 가능성이 제기된다. 국내 시멘트 산업이 '저성장 고비용'의 구조적 위험에 빠진데 따른 영향이라는 분석이다. 다만 한일시멘트를 제외한 주요 기업들은 우선 신중모드로 접근할 전망이다.

20일 관련 업계에 따르면 한일시멘트는 최근 이사회를 열고 77.78% 지분을 보유한 한일현대시멘트를 오는 11월 1일부로 흡수 합병하기로 의결했다. 양사는 2017년부터 단계적으로 통합 작업을 진행해왔다. 이번 합병으로 상장사 이중 구조를 해소하고 중복투자·외부비용 절감을 통한 경영 효율화에 나선다는 방침이다. 한일시멘트 관계자는 "경기가 부침이 있는 만큼 설비 등 인프라 활용해 탄력적 대응이 가능할 것"이라고 말했다.

이미 시멘트 업계의 경영상황은 악화일로다. 국토교통부에 따르면 지난 5월 건설착공면적은 전월 대비 26.9% 감소한 620만㎡ 수준에 그쳤다. 착공면적은 통상 건설경기의 선행지표로 여겨진다. 시멘트 출하량도 급감했다. 올해 1·4분기 시멘트 내수는 전년 동기 대비 21.8% 줄어든 812만t으로, 최근 5년 내 최저치를 기록했다. 2023년 1·4분기 1201만t 대비 2년 새 32.4%가 줄어든 셈이다.

그럼에도 업계 전반의 대응은 신중한 모습이다. 한일을 제외한 주요 업체들은 합병 혹은 조직개편 계획 여부 등은 없다는 입장이다. 쌍용C&E 관계자는 "경기가 좋지 않지만 수출 덕분에 그나마 나은 상황으로 당분간 특별한 계획 없다"고 말했다. 성신양회, 한라시멘트 관계자 역시 "거래처 및 비용 절감에 집중하고 있을 뿐, 지배구조 개편 등은 고려하고 있지 않다"고 했다.

업계는 현재 위기를 수요 부진과 고정비 부담의 이중고로 보고 있다. 시멘트 산업은 설비 비중이 높아 수요가 줄어도 일정 수준의 가동과 인력 유지를 지속해야 하기 때문이다. 게다가 최근 몇 년간 산업용 전기요금 인상, 탄소배출권 비용 증가, 폐기물 처리단가 상승 등 외부 비용도 커졌다.

이 가운데 업계는 환경·사회·지배구조(ESG) 전환에도 동시에 대응 중이다. 시멘트 제조 시 온실가스 배출량이 많다는 점에서, 탄소중립 압력이 거세져 온 탓이다. 이에 오는 2030년까지 2018년 대비 12%, 2050년까지 53%의 온실가스 감축을 목표로 설정하고 있다. 이를 위해 석회석 대신 비탄산염 원료를 활용한 저탄소 클링커 기술, 폐합성수지·바이오매스 등 순환자원 연료 전환 기술 개발 등에 집중하고 있다.

한일시멘트의 이번 결정은 불황기의 생존 해법으로 지배구조 단순화 및 비용 최적화의 가능성을 제기했다는 분석이 나온다.

업계 관계자는 "내수 의존도가 큰 기업일수록 버틸 수 있는 시간이 길지 않다"며 "누가 먼저 구조를 바꾸고, 얼마나 오래버티느냐가 관건"이라고 말했다.
    """,
    
    # 3. 증권 카테고리
    """ (서울=연합뉴스) 임화섭 기자 = 인공지능(AI) 분야 경쟁으로 '매그니피센트 세븐'(미국의 7대 테크 대기업·The Magnificent Seven) 사이에서도 주가 차별화가 심화되고 있다고 미국 일간 월스트리트저널(WSJ)이 20일(현지시간) 분석했다.

테크업계를 주름잡는 대기업들이며 각종 주가지수에서도 큰 비중을 차지하는 아마존닷컴(이하 아마존), 알파벳, 애플, 메타플랫폼스(이하 메타), 마이크로소프트(이하 MS), 엔비디아, 테슬라 등 7개 기업의 주가 흐름이 AI 분야 성과에 따라 크게 갈리는 경향이 최근 뚜렷해졌다는 것이다.

대표적인 주가지수인 S&P 500에서 이들 7개 업체가 차지하는 비중은 35%에 이른다.

최근 모건스탠리 보고서에 따르면 이 업체들의 2분기 순이익은 전년 동기 대비 14% 증가할 것으로 전망되며, 이는 나머지 493개 S&P 500 기업의 순이익이 전년 동기 대비 3% 감소할 것으로 예상되는 전반적 증시 분위기와 비교하면 훨씬 좋다.

이들 중 알파벳을 제외한 6개 업체들은 최근 주가가 향후 1년간 예상 순이익의 25배를 넘었으며, 이는 최근 S&P 500 기업의 평균 주가수익비율(PER) 22.35보다 높다.

올해 들어 엔비디아, 메타, MS의 주가는 약 20% 이상 올랐으나, 이와 대조적으로 애플은 16% 하락했고 알파벳도 2% 내렸고 테슬라는 18%가 빠졌다.

AI 스타트업 '앤스로픽'에 투자한 아마존은 관세 등의 영향으로 전자상거래 부문 사업에 타격이 상당히 큰 가운데서도 3% 올랐다.

알파벳과 테슬라는 오는 23일에, 메타, MS, 애플은 그 다음 주에 분기 실적을 발표할 예정이다.

해리스 파이낸셜 그룹의 매니징 파트너인 제이미 콕스는 WSJ에 "(테크 대기업들 사이에서 실적에 따라 주가 흐름이 차별화되는 것은) 불가피한 일이었다. 다른 일들을 하고 있기 때문에 영원히 서로 똑같은 (주가) 흐름이 계속되는 것은 불가능하다"며 "이제 승자와 패자의 계층분화가 왔다"고 평가했다.

2023년에 이 7개 업체를 묶어서 '매그니피센트 세븐'이라는 표현을 쓰기 시작한 뱅크오브아메리카 글로벌 리서치의 투자전략담당 책임자 마이클 하트넛은 이들 업체를 묶어서 본 중요한 이유 중 하나는 이들이 AI 분야의 선봉에 있기 때문이라고 말한 바 있다.

그러나 투자회사 스피어의 창립자 겸 투자책임자인 이바나 델레브스카는 "지금은 펀더멘털에서 (7개 업체들 사이에) 상당히 큰 차이가 보이고 있다"고 말했다.
    """
]
def generate_output(model, text, system_prompt):
    # messages 형식으로 프롬프트 구성 (system + user)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"다음 뉴스 텍스트를 분석해주세요:\n\n{text}"}
    ]
    
    # apply_chat_template 사용 (thinking 끄기: enable_thinking=False)
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # thinking 모드 OFF (여기서 변경!)
    )
    
    # 입력 토큰화
    inputs = tokenizer([prompt_text], return_tensors="pt").to(model.device)
    
    # generate (deterministic으로 설정)
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_length=8192,  # 최대 길이
            temperature=0.5,  # deterministic (랜덤성 제거)
            top_p=1.0,
            do_sample=False   # 샘플링 off
        )
    
    # 출력 디코딩 (thinking이 없을 테니 전체 출력)
    output_ids = generated_ids[0][len(inputs.input_ids[0]):]  # 생성된 부분만
    decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    
    # 만약 thinking 태그가 남아 있으면 파싱 (안전장치, False로 하면 불필요)
    try:
        # </think> 토큰 ID (Qwen 공식: 151668, 모델에 따라 확인 필요)
        think_end_token = 151668  # Qwen의 </think> 토큰 ID
        index = len(output_ids.tolist()) - output_ids.tolist()[::-1].index(think_end_token)
    except ValueError:
        index = 0  # thinking 없으면 0
    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()
    
    # thinking이 있으면 출력, 없으면 content만
    if thinking_content:
        print("주의: thinking content가 남아 있음:", thinking_content)
    
    return content  # 최종 content 반환 (dictionary 예상)

# 5. 3개 데이터로 베이스 모델 generate 실행
for i, text in enumerate(test_texts, 1):
    print(f"\n===== 테스트 데이터 {i} (카테고리 힌트: 부동산/산업/증권) =====\n{text}\n")
    
    # generate 호출 (모델 로드 후 바로 여기서 generate)
    base_output = generate_output(base, text, system_prompt)
    print("----- 베이스 모델 출력 -----\n", base_output)
    
    print("\n===== 테스트 끝 =====\n")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...


===== 테스트 데이터 1 (카테고리 힌트: 부동산/산업/증권) =====
 최대 수혜지역이라고요? 분위기 좀 달라지긴 했지만, 그게 꼭 6·27 대출 규제 때문만은 아니죠."

지난 19일 서울 도봉구 창동역 인근 한 공인중개사는 최근 시장 분위기를 묻는 질문에 이같이 말했다. 도봉구는 창동 차량기지 복합개발, GTX-C 등 대형 교통 호재에 힘입어 올해 들어 아파트값이 상승세로 전환됐고, 일부 단지에서는 실거래가 회복 조짐도 나타나고 있다. 반면 6·27대책 발표 직후 상승세 강했던 금관구(금천·관악·구로)는 매수·매도인 모두 지켜보기에 들어간 모습이다.

■도봉구 상승세…실거래가 회복 조짐

6·27 대출 규제 시행 3주차에 접어들면서 서울 외곽 지역을 중심으로 시장의 온도차가 뚜렷해지고 있다. 한국부동산원 주간 아파트가격 동향에 따르면 7월 2주(15일 기준) 서울 아파트값은 0.19% 올라 전주(0.29%)보다 오름폭이 둔화됐다. 다만 도봉구(0.05%→0.06%)와 중구(0.16%→0.18%)는 상승폭이 소폭 확대됐다. 도봉구의 한 중개업소 관계자는 "급매가 거의 소진된 상태에서 전고점에 근접한 거래도 하나둘 나타나고 있다"며 "전용 84㎡, 165㎡ 등 다양한 평형에 실수요자와 일부 투자자의 문의가 이어지고 있다"고 전했다. 이어 "6·27 대책 이후 도봉구처럼 상대적으로 대출 여건이 유지되는 중저가 지역에 관심이 모이고 있는 건 사실이지만, 지금 시장을 움직이는 건 결국 개발 기대감에 기반한 유동성 있는 수요자들"이라고 설명했다.

서울부동산정보광장에 따르면 도봉구 아파트 거래량은 5월 155건, 6월 289건으로 늘었지만, 7월 들어서는 20일 기준 51건에 머물고 있다. 규제 여파로 거래량은 다소 숨고르기에 들어간 양상이지만 가격 상승세는 계속될 것으로 전망됐다.

도봉구의 또 다른 중개업소 대표는 "이미 작년에 바닥을 찍었다는 인식이 확산되면서 실수요자 문의가 늘고 있고, 호가도 빠르게 회복되고 있다"며 "GTX-C, 창동 차량기지 개발

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----- 베이스 모델 출력 -----
 {"category": "산업", "event_count": 4, "events": ["6·27 대출 규제 시행 후 도봉구 아파트 가격 상승세 유지", "도봉구 실거래가 회복 조짐 나타나고 있다", "금관구 거래량 감소 및 가격 상승세 하락", "부동산업계 전문가가 외곽 지역 시장 흐름의 불확실성 강조"]}

===== 테스트 끝 =====


===== 테스트 데이터 2 (카테고리 힌트: 부동산/산업/증권) =====

    한일시멘트가 자회사 한일현대시멘트 흡수 합병을 통해 경영효율화에 나서면서 시멘트업계에 구조·사업개편 가능성이 제기된다. 국내 시멘트 산업이 '저성장 고비용'의 구조적 위험에 빠진데 따른 영향이라는 분석이다. 다만 한일시멘트를 제외한 주요 기업들은 우선 신중모드로 접근할 전망이다.

20일 관련 업계에 따르면 한일시멘트는 최근 이사회를 열고 77.78% 지분을 보유한 한일현대시멘트를 오는 11월 1일부로 흡수 합병하기로 의결했다. 양사는 2017년부터 단계적으로 통합 작업을 진행해왔다. 이번 합병으로 상장사 이중 구조를 해소하고 중복투자·외부비용 절감을 통한 경영 효율화에 나선다는 방침이다. 한일시멘트 관계자는 "경기가 부침이 있는 만큼 설비 등 인프라 활용해 탄력적 대응이 가능할 것"이라고 말했다.

이미 시멘트 업계의 경영상황은 악화일로다. 국토교통부에 따르면 지난 5월 건설착공면적은 전월 대비 26.9% 감소한 620만㎡ 수준에 그쳤다. 착공면적은 통상 건설경기의 선행지표로 여겨진다. 시멘트 출하량도 급감했다. 올해 1·4분기 시멘트 내수는 전년 동기 대비 21.8% 줄어든 812만t으로, 최근 5년 내 최저치를 기록했다. 2023년 1·4분기 1201만t 대비 2년 새 32.4%가 줄어든 셈이다.

그럼에도 업계 전반의 대응은 신중한 모습이다. 한일을 제외한 주요 업체들은 합병 혹은 조직개편 계획 여부 등은 없다는 입장이다. 쌍용C&E 관계자는 "경기가 좋지 않지만 수출 덕분에 그나마 나은

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----- 베이스 모델 출력 -----
 답변:
{"category": "산업", "event_count": 4, "events": ["한일시멘트가 자회사 한일현대시멘트를 흡수 합병 결정", "시멘트 산업의 경영상황 악화", "업계 전반의 대응은 신중한 모습", "시멘트 제조 시 온실가스 배출량 증가로 탄소중립 목표 설정"]}

===== 테스트 끝 =====


===== 테스트 데이터 3 (카테고리 힌트: 부동산/산업/증권) =====
 (서울=연합뉴스) 임화섭 기자 = 인공지능(AI) 분야 경쟁으로 '매그니피센트 세븐'(미국의 7대 테크 대기업·The Magnificent Seven) 사이에서도 주가 차별화가 심화되고 있다고 미국 일간 월스트리트저널(WSJ)이 20일(현지시간) 분석했다.

테크업계를 주름잡는 대기업들이며 각종 주가지수에서도 큰 비중을 차지하는 아마존닷컴(이하 아마존), 알파벳, 애플, 메타플랫폼스(이하 메타), 마이크로소프트(이하 MS), 엔비디아, 테슬라 등 7개 기업의 주가 흐름이 AI 분야 성과에 따라 크게 갈리는 경향이 최근 뚜렷해졌다는 것이다.

대표적인 주가지수인 S&P 500에서 이들 7개 업체가 차지하는 비중은 35%에 이른다.

최근 모건스탠리 보고서에 따르면 이 업체들의 2분기 순이익은 전년 동기 대비 14% 증가할 것으로 전망되며, 이는 나머지 493개 S&P 500 기업의 순이익이 전년 동기 대비 3% 감소할 것으로 예상되는 전반적 증시 분위기와 비교하면 훨씬 좋다.

이들 중 알파벳을 제외한 6개 업체들은 최근 주가가 향후 1년간 예상 순이익의 25배를 넘었으며, 이는 최근 S&P 500 기업의 평균 주가수익비율(PER) 22.35보다 높다.

올해 들어 엔비디아, 메타, MS의 주가는 약 20% 이상 올랐으나, 이와 대조적으로 애플은 16% 하락했고 알파벳도 2% 내렸고 테슬라는 18%가 빠졌다.

AI 스타트업 '앤스로픽'에 투자한 아마존은 관세 등의 영향으로 전자상거래 부문 사업에 타격이 상당히 큰 가운데서도

In [2]:

lora = PeftModel.from_pretrained(base,
                                 "./qwen3-lora-ft-final",
                                 device_map="auto",
                                 torch_dtype="auto")   # safetensors 자동 인식

# 5. 3개 데이터로 베이스 모델 generate 실행
for i, text in enumerate(test_texts, 1):
    print(f"\n===== 테스트 데이터 {i} (카테고리 힌트: 부동산/산업/증권) =====\n{text}\n")
    
    # generate 호출 (모델 로드 후 바로 여기서 generate)
    base_output = generate_output(lora, text, system_prompt)
    print("----- 로라 모델 출력 -----\n", base_output)
    
    print("\n===== 테스트 끝 =====\n")

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



===== 테스트 데이터 1 (카테고리 힌트: 부동산/산업/증권) =====
 최대 수혜지역이라고요? 분위기 좀 달라지긴 했지만, 그게 꼭 6·27 대출 규제 때문만은 아니죠."

지난 19일 서울 도봉구 창동역 인근 한 공인중개사는 최근 시장 분위기를 묻는 질문에 이같이 말했다. 도봉구는 창동 차량기지 복합개발, GTX-C 등 대형 교통 호재에 힘입어 올해 들어 아파트값이 상승세로 전환됐고, 일부 단지에서는 실거래가 회복 조짐도 나타나고 있다. 반면 6·27대책 발표 직후 상승세 강했던 금관구(금천·관악·구로)는 매수·매도인 모두 지켜보기에 들어간 모습이다.

■도봉구 상승세…실거래가 회복 조짐

6·27 대출 규제 시행 3주차에 접어들면서 서울 외곽 지역을 중심으로 시장의 온도차가 뚜렷해지고 있다. 한국부동산원 주간 아파트가격 동향에 따르면 7월 2주(15일 기준) 서울 아파트값은 0.19% 올라 전주(0.29%)보다 오름폭이 둔화됐다. 다만 도봉구(0.05%→0.06%)와 중구(0.16%→0.18%)는 상승폭이 소폭 확대됐다. 도봉구의 한 중개업소 관계자는 "급매가 거의 소진된 상태에서 전고점에 근접한 거래도 하나둘 나타나고 있다"며 "전용 84㎡, 165㎡ 등 다양한 평형에 실수요자와 일부 투자자의 문의가 이어지고 있다"고 전했다. 이어 "6·27 대책 이후 도봉구처럼 상대적으로 대출 여건이 유지되는 중저가 지역에 관심이 모이고 있는 건 사실이지만, 지금 시장을 움직이는 건 결국 개발 기대감에 기반한 유동성 있는 수요자들"이라고 설명했다.

서울부동산정보광장에 따르면 도봉구 아파트 거래량은 5월 155건, 6월 289건으로 늘었지만, 7월 들어서는 20일 기준 51건에 머물고 있다. 규제 여파로 거래량은 다소 숨고르기에 들어간 양상이지만 가격 상승세는 계속될 것으로 전망됐다.

도봉구의 또 다른 중개업소 대표는 "이미 작년에 바닥을 찍었다는 인식이 확산되면서 실수요자 문의가 늘고 있고, 호가도 빠르게 회복되고 있다"며 "GTX-C, 창동 차량기지 개발

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----- 로라 모델 출력 -----
 {"category": "부동산","event_count": 2,"events": ["서울부동산정보광장에 따르면 도봉구 아파트 거래량은 5월 155건, 6월 289건으로 늘었지만, 7월 들어서는 20일 기준 51건에 머물고 있다.", "도봉구의 한 중개업소 대표는 \"이미 작년에 바닥을 찍었다는 인식이 확산되면서 실수요자 문의가 늘고 있고, 호가도 빠르게 회복되고 있다\"며 \"GTX-C, 창동 차량기지 개발 등 향후 호재가 여전한 만큼, 지금은 회복의 시작일 뿐이라는 얘기도 나온다\"고 했다."]}

===== 테스트 끝 =====


===== 테스트 데이터 2 (카테고리 힌트: 부동산/산업/증권) =====

    한일시멘트가 자회사 한일현대시멘트 흡수 합병을 통해 경영효율화에 나서면서 시멘트업계에 구조·사업개편 가능성이 제기된다. 국내 시멘트 산업이 '저성장 고비용'의 구조적 위험에 빠진데 따른 영향이라는 분석이다. 다만 한일시멘트를 제외한 주요 기업들은 우선 신중모드로 접근할 전망이다.

20일 관련 업계에 따르면 한일시멘트는 최근 이사회를 열고 77.78% 지분을 보유한 한일현대시멘트를 오는 11월 1일부로 흡수 합병하기로 의결했다. 양사는 2017년부터 단계적으로 통합 작업을 진행해왔다. 이번 합병으로 상장사 이중 구조를 해소하고 중복투자·외부비용 절감을 통한 경영 효율화에 나선다는 방침이다. 한일시멘트 관계자는 "경기가 부침이 있는 만큼 설비 등 인프라 활용해 탄력적 대응이 가능할 것"이라고 말했다.

이미 시멘트 업계의 경영상황은 악화일로다. 국토교통부에 따르면 지난 5월 건설착공면적은 전월 대비 26.9% 감소한 620만㎡ 수준에 그쳤다. 착공면적은 통상 건설경기의 선행지표로 여겨진다. 시멘트 출하량도 급감했다. 올해 1·4분기 시멘트 내수는 전년 동기 대비 21.8% 줄어든 812만t으로, 최근 5년 내 최저치를 기록했다. 2023년 1·4분기 1201만t 대비 2년 새 32.4%가 줄어든 셈이다.


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


----- 로라 모델 출력 -----
 {"category": "산업","event_count": 2,"events": ["20일 관련 업계에 따르면 한일시멘트는 최근 이사회를 열고 77.78% 지분을 보유한 한일현대시멘트를 오는 11월 1일부로 흡수 합병하기로 의결했다.", "이미 시멘트 업계의 경영상황은 악화일로다."]}

===== 테스트 끝 =====


===== 테스트 데이터 3 (카테고리 힌트: 부동산/산업/증권) =====
 (서울=연합뉴스) 임화섭 기자 = 인공지능(AI) 분야 경쟁으로 '매그니피센트 세븐'(미국의 7대 테크 대기업·The Magnificent Seven) 사이에서도 주가 차별화가 심화되고 있다고 미국 일간 월스트리트저널(WSJ)이 20일(현지시간) 분석했다.

테크업계를 주름잡는 대기업들이며 각종 주가지수에서도 큰 비중을 차지하는 아마존닷컴(이하 아마존), 알파벳, 애플, 메타플랫폼스(이하 메타), 마이크로소프트(이하 MS), 엔비디아, 테슬라 등 7개 기업의 주가 흐름이 AI 분야 성과에 따라 크게 갈리는 경향이 최근 뚜렷해졌다는 것이다.

대표적인 주가지수인 S&P 500에서 이들 7개 업체가 차지하는 비중은 35%에 이른다.

최근 모건스탠리 보고서에 따르면 이 업체들의 2분기 순이익은 전년 동기 대비 14% 증가할 것으로 전망되며, 이는 나머지 493개 S&P 500 기업의 순이익이 전년 동기 대비 3% 감소할 것으로 예상되는 전반적 증시 분위기와 비교하면 훨씬 좋다.

이들 중 알파벳을 제외한 6개 업체들은 최근 주가가 향후 1년간 예상 순이익의 25배를 넘었으며, 이는 최근 S&P 500 기업의 평균 주가수익비율(PER) 22.35보다 높다.

올해 들어 엔비디아, 메타, MS의 주가는 약 20% 이상 올랐으나, 이와 대조적으로 애플은 16% 하락했고 알파벳도 2% 내렸고 테슬라는 18%가 빠졌다.

AI 스타트업 '앤스로픽'에 투자한 아마존은 관세 등의 영향으로 전자상거래 부문 사업에 타격이 상당히 큰 가운데서도 